In [14]:
# 구글 드라이브와 연동합니다
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Package 설치

In [30]:
# 필요한 모듈을 설치합니다
!pip install -q transformers pytorch_lightning emoji soynlp

# 데이터 불러오기, 데이터 전처리

In [31]:
import pandas as pd

sad = pd.read_excel("/content/drive/My Drive/data/tweet_list_슬픔 1_5000.xlsx")
happy = pd.read_excel("/content/drive/My Drive/data/tweet_list_기쁨 labeling_완료.xlsx")
annoy = pd.read_excel('/content/drive/My Drive/data/tweet_list_짜증_완료.xlsx')
fear = pd.read_excel('/content/drive/My Drive/data/tweet_list_공포_완료.xlsx')

sad2 = pd.read_csv('/content/drive/My Drive/data/추가_슬픔.csv')
happy2 = pd.read_csv('/content/drive/My Drive/data/추가_기쁨.csv')
annoy2 = pd.read_csv('/content/drive/My Drive/data/추가_분노.csv')
fear2 = pd.read_csv('/content/drive/My Drive/data/추가_공포1.txt', encoding='utf8')

sad

,Unnamed: 0,raw_text
0,5610,가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며...
1,2189,가담도 완결나면 유료화되겠지 좀 슬픔
2,1843,가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠㅠㅠ
3,497,가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되...
4,2165,가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔
...,...,...
2691,7241,오페라의 유령 공연까지 일 남았다 근데 그때면 개강도 머지않아서 슬픔
2692,4421,그런것같아ㅋㅋㅋㅋ 첨에는 목아프더니 갑자기 콧물나오고 장난아니랄까 목 다갈라쟛어 슬픔
2693,4862,그런데 왜 입술을 꾹 물고 슬픔에 물든 눈으로 너를 본다 왜 그렇게 말해
2694,26,언니도 같이 달려줫던 년 좀 슬픔


In [32]:
#전처리를 위한 함수
def preprocessing(data, label):
  import re
  dt = data['raw_text'].copy() #문장만 선택
  dt = dt.dropna() #결측치 제거
  dt = dt.drop_duplicates() #중복 제거
  sentences = dt.tolist()
  new_sent=[]
  for i in range(len(sentences)):
    sent = sentences[i] 
    if type(sent) != str: # 문장 중 str 아닌 것 처리
      sent = str(sent)
    if len(sent) < 2: continue #길이 1 이상인 것만 선택
    sent = re.sub('ㅋㅋ+','ㅋㅋ',sent)
    sent = re.sub('ㅠㅠ+','ㅠㅠ',sent)
    sent = re.sub('ㅇㅇ+','ㅇㅇ',sent) 
    sent = re.sub('ㄷㄷ+','ㄷㄷ',sent) 
    sent = re.sub('ㅎㅎ+','ㅎㅎ',sent)
    sent = re.sub('ㅂㅂ+','ㅂㅂ',sent) 
    sent = re.sub(';;;+',';;',sent)
    sent = re.sub('!!!+','!!',sent)
    sent = re.sub('~+','~',sent)
    sent = re.sub('[?][?][?]+','??',sent) 
    sent = re.sub('[.][.][.]+','...',sent) 
    sent = re.sub('[-=+,#/:$@*\"※&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]','',sent)
    new_sent.append(sent)
  dt = pd.DataFrame(pd.Series(new_sent), columns=['raw_text'])
  dt['emotion'] = label
  return dt

In [33]:
sad = preprocessing(sad, '슬픔')
sad2 = preprocessing(sad2, '슬픔')
happy = preprocessing(happy, '기쁨')
happy2 = preprocessing(happy2, '기쁨')
annoy = preprocessing(annoy, '분노')
annoy2 = preprocessing(annoy2, '분노')
fear = preprocessing(fear, '공포')
fear2 = preprocessing(fear2, '공포')

for i in [sad, happy, annoy, fear]:
  print('1차 레이블 결과', i['emotion'][0],len(i))
for i in [sad2, happy2, annoy2, fear2]:
  print('2차 레이블 결과', i['emotion'][0],len(i))

print('최소 데이터: 공포 ', len(fear)+len(fear2))

1차 레이블 결과 슬픔 2523
1차 레이블 결과 기쁨 2064
1차 레이블 결과 분노 7862
1차 레이블 결과 공포 1034
2차 레이블 결과 슬픔 2931
2차 레이블 결과 기쁨 3569
2차 레이블 결과 분노 1967
2차 레이블 결과 공포 3280
최소 데이터: 공포  4314


In [34]:
## 데이터 개수 확인 후 학습을 위해 각 감정별 데이터 개수를 동일하게 맞춰줍니다.
sad_3 = sad[:1400]
happy_3 = happy[:800]
annoy_3 = annoy[:2400]

# 각 감정별 키워드 데이터가 약 1000개 씩으로 이루어져 있기 때문에 마지막 키워드에 대한 데이터 1000개를 평가 데이터로 선택
sentence_train = pd.concat([sad_3, happy_3, annoy_3, fear, sad2[:-1000], annoy2[:-1000], happy2[:-1000], fear2[:-1000]], axis=0, ignore_index=True)
sentence_eval = pd.concat([sad2[-1000:], annoy2[-1000:], happy2[-1000:], fear2[-1000:]], axis=0, ignore_index=True)

for i in ['슬픔','기쁨','분노','공포']:
  print('sentence_train',i,len(sentence_train[sentence_train['emotion'] == i]))
print('-------------------------')
for i in ['슬픔','기쁨','분노','공포']:
  print('sentence_eval',i,len(sentence_eval[sentence_eval['emotion'] == i]))

sentence_train 슬픔 3331
sentence_train 기쁨 3369
sentence_train 분노 3367
sentence_train 공포 3314
-------------------------
sentence_eval 슬픔 1000
sentence_eval 기쁨 1000
sentence_eval 분노 1000
sentence_eval 공포 1000


In [35]:
#모델에 입력하기 위해 형식을 맞춰줍니다
def label(x):
  if x=='슬픔': return 0
  elif x=='기쁨': return 1
  elif x=='분노': return 2
  elif x=='공포': return 3
  else: return x

sentence_train["emotion"] = sentence_train["emotion"].apply(label)

dtls = [list(sentence_train.iloc[i,:]) for i in range(len(sentence_train))]
dtls[:10] #형식이 통일되었습니다
#dtls.to_csv("/content/drive/My Drive/data/emotion_train.csv")

[['가난한 아버지가 가련한 아들을 껴안고 잠든 밤 마른 이불과 따끈따끈한 요리를 꿈꾸며 잠든 밤 큰 슬픔이 작은 슬픔을 껴안고 잠든 밤 소금 같은 싸락눈이 신문지 갈피를 넘기며 염장을 지르는 지하역의 겨울밤박후기자반고등어',
  0],
 ['가담도 완결나면 유료화되겠지 좀 슬픔', 0],
 ['가려진시간 뭔데 존나 슬픔 하아 ㅠㅠ 아 시발 진짜 시발 ㅠㅠ', 0],
 ['가리고 싶은 감정이 참 많았어요 뭐라 부르기도 싫고 그냥 그런 것이 없는 사람이 되고 싶었어요 제 안에 긍정적이고 밝은 마음만 있기를 바랬나 봐요 그래도 슬픔은 드러내기 어렵지 않았어요 그건 누구에게나 이해받을 수 있다고 생각했지만 미련 유치함 변덕 등은 모두 없었으면 했어요',
  0],
 ['가만보면 블루님은 다 귀엽다고 하시는데 본인이 귀여운지는 모르시는 거 같아요 슬픔', 0],
 ['가만히있던 슬픔 띠용 ', 0],
 ['가발도 구했고 안구도왔구 이제 메이크업만 하면 완성인데 기다리던 분이 한동안 쉬신대슬픔', 0],
 ['가방 속에 넣어두었던 돈일반적인 고액은 아니지만 나에게는 고액봉투 사라져서 슬픔 ㅅㅂ 얘야 어디갔ㅇ니 으흑', 0],
 ['가버리셨나슬픔에 철푸덕 쓰러짐', 0],
 ['가보고싶은 맛집은 참 많은데 인 이상가야 본전 뽑는곳이 너무 많아서 슬픔 ', 0]]

In [36]:
# 모델에 입력하기 위한 형태로 train 데이터 저장
with open("/content/drive/My Drive/data/emotion_train.txt", "w") as file:
  file.write("id\tdocument\tlabel\n")
  for idx, slist in enumerate(dtls):
    file.write("{}\t{}\t{}\n".format(idx, slist[0], slist[1]))

file = open("/content/drive/My Drive/data/emotion_train.txt", "r")

In [37]:
##################################################
# 평가용 Test_set을 모델에 입력하기 위해 형식을 맞춰줍니다

sentence_eval["emotion"] = sentence_eval["emotion"].apply(label)

dtls_eval = [list(sentence_eval.iloc[i,:]) for i in range(len(sentence_eval))]

In [38]:
# 모델에 입력하기 위한 형태로 eval 데이터 저장
with open("/content/drive/My Drive/data/emotion_eval.txt", "w") as file:
  file.write("id\tdocument\tlabel\n")
  for idx, slist in enumerate(dtls_eval):
    file.write("{}\t{}\t{}\n".format(idx, slist[0], slist[1]))

file = open("/content/drive/My Drive/data/emotion_eval.txt", "r")

# Package import & 기본 Args 설정

In [15]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import BertForSequenceClassification, BertTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

# 기본 학습 Arguments

In [16]:
class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-large'  # Transformers PLM name
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size 
    batch_size: int = 0  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size` 
    lr: float = 5e-6  # Starting Learning Rate
    epochs: int = 20  # Max Epochs
    max_length: int = 150  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    train_data_path: str = "/content/drive/My Drive/data/emotion_train.txt"  # Train Dataset file 
    val_data_path: str = "/content/drive/My Drive/data/emotion_eval.txt"  # Validation Dataset file 
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores

args = Arg()

In [17]:
##GPU 사용 시
device = torch.device("cuda:0")

## 기본값을 Override 하고싶은 경우 아래와 같이 수정

In [26]:
# args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 16  # Force setup batch_size

# Model 만들기 with Pytorch Lightning

In [23]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        self.args = options
        self.bert = BertForSequenceClassification.from_pretrained(self.args.pretrained_model)
        self.tokenizer = BertTokenizer.from_pretrained(
            self.args.pretrained_tokenizer
            if self.args.pretrained_tokenizer
            else self.args.pretrained_model
        )

    def forward(self, **kwargs):
        return self.bert(**kwargs)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        loss, logits = self(input_ids=data, labels=labels)
        preds = logits.argmax(dim=-1)

        y_true = labels.cpu().numpy()
        y_pred = preds.cpu().numpy()

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'train_loss': loss.cpu().detach().numpy().tolist(),
            'train_acc': metrics[0],
            'train_precision': metrics[1],
            'train_recall': metrics[2],
            'train_f1': metrics[3],
        }
        if (batch_idx % self.args.report_cycle) == 0:
            print()
            pprint(tensorboard_logs)
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        loss, logits = self(input_ids=data, labels=labels)
        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def validation_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[1],
            'val_recall': metrics[2],
            'val_f1': metrics[3],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        if self.args.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.args.lr)
        elif self.args.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.args.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.args.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.args.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def preprocess_dataframe(self, df):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

        def clean(x):
            x = pattern.sub(' ', x)
            x = url_pattern.sub('', x)
            x = x.strip()
            x = repeat_normalize(x, num_repeats=2)
            return x

        df['document'] = df['document'].map(lambda x: self.tokenizer.encode(
            clean(str(x)),
            padding='max_length',
            max_length=self.args.max_length,
            truncation=True,
        ))
        return df

    def train_dataloader(self):
        df = self.read_data(self.args.train_data_path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )

    def val_dataloader(self):
        df = self.read_data(self.args.val_data_path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )

In [24]:
def main():
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", args.random_seed)
    seed_everything(args.random_seed)
    model = Model(args)

    print(":: Start Training ::")
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        auto_scale_batch_size=args.auto_batch_size if args.auto_batch_size and not args.batch_size else False,
        # For GPU Setup
        deterministic=torch.cuda.is_available(),
        gpus=1 if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        # For TPU Setup
        # tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )
    trainer.fit(model)

# Train
- 주의 : 1epoch별로 GPU-P100 기준 약 2~3시간, GPU V100기준 ~40분이 걸립니다.
- Update @2020.09.02 최근 Colab Pro에서 V100이 배정됩니다.
```python
# 1epoch 기준 아래 score가 나옵니다.
{'val_acc': 0.90522,
 'val_f1': 0.9049023739289227,
 'val_loss': 0.23429009318351746,
 'val_precision': 0.9143146796431468,
 'val_recall': 0.8956818813808446}
 ```

In [25]:
main()
# CUDA error: device-side assert triggered << 현재 수정 중

Using PyTorch Ver 1.6.0+cu101
Fix Seed: 42


Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

:: Start Training ::


RuntimeError: ignored

# Test